In [1]:
import os
import time

import pyreadr as py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, RMSprop
from torch.optim.lr_scheduler import StepLR

from sklearn.model_selection import train_test_split

torch.manual_seed(42)
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

cuda:3


> ### Data downloading
Data link  
https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/6C3JR1

##### Data description
Here we consoder dataset of "Additional Tennessee Eastman Process Simulation Data for Anomaly Detection Evaluation"
This dataverse contains the data referenced in Rieth et al. (2017). Issues and Advances in Anomaly Detection Evaluation for Joint Human-Automated Systems. To be presented at Applied Human Factors and Ergonomics 2017.
##### Columns description
* **faultNumber** ranges from 1 to 20 in the “Faulty” datasets and represents the fault type in the TEP. The “FaultFree” datasets only contain fault 0 (i.e. normal operating conditions).
* **simulationRun** ranges from 1 to 500 and represents a different random number generator state from which a full TEP dataset was generated (Note: the actual seeds used to generate training and testing datasets were non-overlapping).
* **sample** ranges either from 1 to 500 (“Training” datasets) or 1 to 960 (“Testing” datasets). The TEP variables (columns 4 to 55) were sampled every 3 minutes for a total duration of 25 hours and 48 hours respectively. Note that the faults were introduced 1 and 8 hours into the Faulty Training and Faulty Testing datasets, respectively.
* **columns 4-55** contain the process variables; the column names retain the original variable names.

In [2]:
# ! unzip dataverse_files.zip -d dataverse_files

In [3]:
#reading train data in .R format
a1 = py.read_r("dataverse_files/TEP_FaultFree_Training.RData")
a2 = py.read_r("dataverse_files/TEP_Faulty_Training.RData")

In [4]:
#reading test data in .R format
a3 = py.read_r("dataverse_files/TEP_FaultFree_Testing.RData")
a4 = py.read_r("dataverse_files/TEP_Faulty_Testing.RData")

In [5]:
print("Objects that are present in a1 :", a1.keys())
print("Objects that are present in a2 :", a2.keys())
print("Objects that are present in a3 :", a3.keys())
print("Objects that are present in a4 :", a4.keys())

Objects that are present in a1 : odict_keys(['fault_free_training'])
Objects that are present in a2 : odict_keys(['faulty_training'])
Objects that are present in a3 : odict_keys(['fault_free_testing'])
Objects that are present in a4 : odict_keys(['faulty_testing'])


In [6]:
# concatinating the train and the test dataset

# train dataframe
raw_train = pd.concat([a1['fault_free_training'], a2['faulty_training']])
# test dataframe
raw_test = pd.concat([a3['fault_free_testing'], a4['faulty_testing']])

In [7]:
raw_train['index'] = raw_train['faultNumber'] * 500 + raw_train['simulationRun'] - 1
raw_test['index'] = raw_test['faultNumber'] * 500 + raw_test['simulationRun'] - 1

> ### EDA

In [8]:
# for col in raw_train.columns[3:]:
#     plt.figure(figsize=(10,5))
#     plt.hist(raw_train[col])
#     plt.xlabel(col)
#     plt.ylabel('counts')
#     plt.show()

In [9]:
# raw_train.groupby(['faultNumber', 'simulationRun']).agg({'xmeas_1': list, 'xmeas_2': list}).reset_index()

In [10]:
features = ['xmeas_1', 'xmeas_2',
       'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8',
       'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14',
       'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20',
       'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26',
       'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32',
       'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38',
       'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4',
       'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11']

In [500]:
def accuracy(y_pred, target):
    
    y_pred = torch.softmax(y_pred, dim=1)
    y_pred = torch.max(y_pred, dim=1)[1]  
    
    return torch.eq(y_pred, target).sum().item()

> ### Sampling

Described in "Data Preparation for Deep Learning Models" in [that article](https://medium.com/@mrunal68/tennessee-eastman-process-simulation-data-for-anomaly-detection-evaluation-d719dc133a7f)

In [11]:
# 5.250.000, 10.080.000
len(raw_train), len(raw_test)

(5250000, 10080000)

In [12]:
simulation_idx = raw_train[['index', 'faultNumber']].drop_duplicates()

In [13]:
X_train_idx, X_val_idx = train_test_split(simulation_idx['index'], 
                                          stratify=simulation_idx['faultNumber'],
                                          test_size=0.2, 
                                          random_state=42)

In [14]:
X_train = raw_train[raw_train['index'].isin(X_train_idx)].drop('index', axis=1)
# y_train = raw_train[raw_train['index'].isin(X_train_idx)][['index', 'faultNumber']]

X_val = raw_train[raw_train['index'].isin(X_val_idx)].drop('index', axis=1)
# y_val = raw_train[raw_train['index'].isin(X_val_idx)][['index', 'faultNumber']]

# X_test = raw_test.drop(['faultNumber', 'simulationRun', 'sample'], axis=1)
# y_test = raw_test['faultNumber']

In [700]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
scaler.fit(X_train[features])

X_train[features] = scaler.transform(X_train[features])
X_val[features] = scaler.transform(X_val[features])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [715]:
# X_raw_train = raw_train[raw_train['faultNumber'].isin([0, 1, 10, 20])].drop('index', axis=1)
X_raw_train = raw_train.drop('index', axis=1)
len(X_raw_train)

5250000

In [716]:
scaler_raw = StandardScaler()
scaler_raw.fit(X_raw_train[features])

X_raw_train[features] = scaler_raw.transform(X_raw_train[features])

In [754]:
class DataTEP(Dataset):

    def __init__(self, X):
    
        self.X = X
        self.X = self.X.sort_values(['faultNumber', 'simulationRun', 'sample'])
        self.X['index'] = self.X.groupby(['faultNumber', 'simulationRun']).ngroup()
        self.X = self.X.set_index('index')

    def __len__(self):
        return self.X.index.max() + 1
    
    def __getitem__(self, idx):
        
        features = [
       'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8',
       'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14',
       'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20',
       'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26',
       'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32',
       'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38',
       'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4',
       'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11']
        
        features = self.X.loc[idx][features].values
        target = self.X.loc[idx]['faultNumber'].unique()[0]
        
#         features = torch.tensor(features, dtype=torch.float)[-10:]
#         features = torch.tensor(features, dtype=torch.float)[:, 0].unsqueeze(-1)
#         features = torch.tensor(features, dtype=torch.float)[-50:, :30]
        features = torch.tensor(features, dtype=torch.float)[-50:, :]
#         features = torch.tensor(features, dtype=torch.float)[:, 1].unsqueeze(-1)
        target = torch.tensor(target, dtype=torch.long)

        return features, target

In [755]:
import gc
gc.collect()

158

In [756]:
BATCH_SIZE = 64
NUM_CLASSES = 21
NUM_EPOCHS = 120
LEARNING_RATE = 0.001

In [757]:
raw_ds = DataTEP(X_raw_train)
raw_dl = DataLoader(raw_ds, batch_size=64, shuffle=True)

train_ds = DataTEP(X_train)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

val_ds = DataTEP(X_val)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE*4, shuffle=True)

# test_ds = DataTEP(X_test, y_test)
# test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE*4)

In [758]:
def check_dl(dl):
    for (X_batch, y_batch) in tqdm(dl):
        print("min", y_batch.numpy().min())
        print("max", y_batch.numpy().max())
        print(X_batch.size(), y_batch.size())
#         print(np.bincount(y_batch.numpy()))

# check_dl(raw_dl)

```python
model_1 = Sequential()
model_1.add(LSTM(256, input_shape=(52, 1), return_sequences=True))
model_1.add(LSTM(128, return_sequences=False))
model_1.add(Dense(300))
model_1.add(Dropout(0.5))
model_1.add(Dense(128))
model_1.add(Dense(21, activation='softmax'))
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_1.summary())
```

In [761]:
class Model(torch.nn.Module) :
    def __init__(self, NUM_LAYERS, INPUT_SIZE, HIDDEN_SIZE, LINEAR_SIZE, OUTPUT_SIZE, DEVICE):
        super().__init__()
        
        self.hidden_size = HIDDEN_SIZE
        self.num_layers = NUM_LAYERS
        self.input_size = INPUT_SIZE
        self.linear_size = LINEAR_SIZE
        self.output_size = OUTPUT_SIZE
        self.directions = True
        
        self.lstm = nn.LSTM(
                        input_size=self.input_size, 
                        hidden_size=self.hidden_size,
                        num_layers=self.num_layers, 
                        bidirectional=self.directions,
                        batch_first=True,
                        dropout=0.4
            )
        
        self.head = nn.Sequential(
                        nn.Linear(in_features=self.hidden_size*(self.directions+1), out_features=self.linear_size),
                        nn.ReLU(),
                        nn.Dropout(p=0.4),
                        nn.Linear(in_features=self.linear_size, out_features=OUTPUT_SIZE),
            )
        
    def forward(self, x):
        
        x, _ = self.lstm(x)
        x = self.head(x[:, -1])
        
        return x

In [765]:
model = Model(NUM_LAYERS=2, INPUT_SIZE=52, HIDDEN_SIZE=128, LINEAR_SIZE=128, OUTPUT_SIZE=NUM_CLASSES, DEVICE=device)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=30, gamma=0.5)

In [766]:
for i, (X_batch_train, y_batch_train) in enumerate(raw_dl):
    if i < 3:  
        X_batch_train, y_batch_train = X_batch_train.to(device), y_batch_train.to(device)
        y_pred_train = model(X_batch_train)
        print("y_batch_train.size()", y_batch_train.size())
        print("y_pred_train.size()", y_pred_train.size(), '\n')
    else:
        break

y_batch_train.size() torch.Size([64])
y_pred_train.size() torch.Size([64, 21]) 

y_batch_train.size() torch.Size([64])
y_pred_train.size() torch.Size([64, 21]) 

y_batch_train.size() torch.Size([64])
y_pred_train.size() torch.Size([64, 21]) 



In [767]:
loss_train_all, loss_val_all = [], []
accuracy_train_all, accuracy_val_all = [], []

for epoch in range(NUM_EPOCHS):

    start = time.time()
    print(f'Epoch: {epoch}, Learning Rate: {scheduler.get_last_lr()[0]}\n')

    loss_epoch_train, loss_epoch_val = 0, 0
    correct_train, correct_val, total_train, total_val = 0, 0, 0, 0

    model.train()
    for (X_batch_train, y_batch_train) in tqdm(train_dl):

        X_batch_train, y_batch_train = X_batch_train.to(device), y_batch_train.to(device)

        optimizer.zero_grad()

        y_pred_train = model(X_batch_train)
        loss_train = criterion(y_pred_train, y_batch_train)

        loss_train.backward()
        optimizer.step()

        loss_epoch_train += loss_train.item()
        correct_train += accuracy(y_pred_train, y_batch_train)
        total_train += y_batch_train.size()[0]

    scheduler.step()
    model.eval()

    with torch.no_grad():
        
        for item in model.parameters():
            print(item.grad.mean())
        
        for (X_batch_val, y_batch_val) in tqdm(val_dl):

            X_batch_val, y_batch_val = X_batch_val.to(device), y_batch_val.to(device)

            y_pred_val = model(X_batch_val)
#                 print(y_pred_val)
            loss_val = criterion(y_pred_val, y_batch_val)

            loss_epoch_val += loss_val.item()

            correct_val += accuracy(y_pred_val, y_batch_val)
            total_val += y_batch_val.size()[0]


    loss_train_tmp = loss_epoch_train / len(train_dl)
    loss_val_tmp = loss_epoch_val / len(val_dl)

    loss_train_all.append(loss_train_tmp)
    loss_val_all.append(loss_val_tmp)

    accuracy_train_tmp = correct_train / total_train
    accuracy_val_tmp = correct_val / total_val

    accuracy_train_all.append(accuracy_train_tmp)
    accuracy_val_all.append(accuracy_val_tmp)

    end = time.time()
    print(f"epoch time: {end - start}")  
    print(f"loss train: {loss_train_tmp}, loss val: {loss_val_tmp}")
    print(f"accuracy train: {accuracy_train_tmp}, accuracy val: {accuracy_val_tmp}")

    print("---------------------------------------------------------------------------------------------------")

Epoch: 0, Learning Rate: 0.001




tensor(-8.1929e-06, device='cuda:3')
tensor(-4.0269e-06, device='cuda:3')
tensor(-0.0012, device='cuda:3')
tensor(-0.0012, device='cuda:3')
tensor(-6.2804e-06, device='cuda:3')
tensor(8.2286e-07, device='cuda:3')
tensor(0.0003, device='cuda:3')
tensor(0.0003, device='cuda:3')
tensor(1.2708e-07, device='cuda:3')
tensor(-8.7884e-07, device='cuda:3')
tensor(0.0001, device='cuda:3')
tensor(0.0001, device='cuda:3')
tensor(6.4294e-08, device='cuda:3')
tensor(0., device='cuda:3')
tensor(-7.8032e-05, device='cuda:3')
tensor(-7.8032e-05, device='cuda:3')
tensor(9.1040e-07, device='cuda:3')
tensor(-0.0013, device='cuda:3')
tensor(-6.6834e-10, device='cuda:3')
tensor(0., device='cuda:3')



epoch time: 26.393052577972412
loss train: 2.0110597980744913, loss val: 1.658794690580929
accuracy train: 0.34, accuracy val: 0.43761904761904763
---------------------------------------------------------------------------------------------------
Epoch: 1, Learning Rate: 0.001




tensor(-1.1022e-05, device='cuda:3')
tensor(1.2160e-06, device='cuda:3')
tensor(-0.0002, device='cuda:3')
tensor(-0.0002, device='cuda:3')
tensor(-1.0699e-05, device='cuda:3')
tensor(7.7189e-07, device='cuda:3')
tensor(7.2190e-05, device='cuda:3')
tensor(7.2190e-05, device='cuda:3')
tensor(-2.7187e-08, device='cuda:3')
tensor(1.3837e-06, device='cuda:3')
tensor(1.3450e-05, device='cuda:3')
tensor(1.3450e-05, device='cuda:3')
tensor(-8.5050e-08, device='cuda:3')
tensor(0., device='cuda:3')
tensor(-2.4738e-05, device='cuda:3')
tensor(-2.4738e-05, device='cuda:3')
tensor(-1.2742e-05, device='cuda:3')
tensor(-0.0005, device='cuda:3')
tensor(-4.5279e-09, device='cuda:3')
tensor(-4.5236e-09, device='cuda:3')



epoch time: 26.222947120666504
loss train: 1.4812517635750049, loss val: 1.3345267632428337
accuracy train: 0.4717857142857143, accuracy val: 0.5085714285714286
---------------------------------------------------------------------------------------------------
Epoch: 2, Learning Rate: 0.001



...


epoch time: 26.547446727752686
loss train: 0.0944608777592128, loss val: 0.2272898142828661
accuracy train: 0.9519047619047619, accuracy val: 0.8952380952380953
---------------------------------------------------------------------------------------------------
Epoch: 118, Learning Rate: 0.000125




tensor(1.1437e-09, device='cuda:3')
tensor(-4.1495e-11, device='cuda:3')
tensor(-6.6422e-08, device='cuda:3')
tensor(-6.6422e-08, device='cuda:3')
tensor(5.9061e-10, device='cuda:3')
tensor(6.5394e-11, device='cuda:3')
tensor(-7.2110e-09, device='cuda:3')
tensor(-7.2110e-09, device='cuda:3')
tensor(-7.8521e-11, device='cuda:3')
tensor(1.3441e-09, device='cuda:3')
tensor(-5.0532e-08, device='cuda:3')
tensor(-5.0532e-08, device='cuda:3')
tensor(-7.7318e-11, device='cuda:3')
tensor(0., device='cuda:3')
tensor(-1.3759e-08, device='cuda:3')
tensor(-1.3759e-08, device='cuda:3')
tensor(1.1390e-09, device='cuda:3')
tensor(-1.5666e-07, device='cuda:3')
tensor(4.6601e-09, device='cuda:3')
tensor(2.6803e-09, device='cuda:3')



epoch time: 27.533787727355957
loss train: 0.09566114333486467, loss val: 0.23135405976106138
accuracy train: 0.9514285714285714, accuracy val: 0.9
---------------------------------------------------------------------------------------------------
Epoch: 119, Learning Rate: 0.000125




tensor(2.3460e-06, device='cuda:3')
tensor(-1.4940e-07, device='cuda:3')
tensor(2.4989e-05, device='cuda:3')
tensor(2.4989e-05, device='cuda:3')
tensor(-9.8617e-07, device='cuda:3')
tensor(7.9953e-08, device='cuda:3')
tensor(-2.2451e-05, device='cuda:3')
tensor(-2.2451e-05, device='cuda:3')
tensor(7.4203e-08, device='cuda:3')
tensor(3.3106e-07, device='cuda:3')
tensor(-2.2226e-05, device='cuda:3')
tensor(-2.2226e-05, device='cuda:3')
tensor(-8.3354e-09, device='cuda:3')
tensor(0., device='cuda:3')
tensor(-8.5520e-06, device='cuda:3')
tensor(-8.5520e-06, device='cuda:3')
tensor(1.3590e-07, device='cuda:3')
tensor(-1.3366e-06, device='cuda:3')
tensor(1.0213e-09, device='cuda:3')
tensor(-2.2174e-11, device='cuda:3')



epoch time: 26.848471641540527
loss train: 0.11444726302711801, loss val: 0.25066431918564963
accuracy train: 0.9432142857142857, accuracy val: 0.8933333333333333
---------------------------------------------------------------------------------------------------


In [ ]:
# accuracy train: 0.9520238095238095, accuracy val: 0.900952380952381 256 хидден фичей

In [ ]:
# accuracy train: 0.9517857142857142, accuracy val: 0.909047619047619 128 хидден фичей

In [ ]:
# accuracy train: 0.7132142857142857, accuracy val: 0.7028571428571428

In [ ]:
# accuracy train: 0.641547619047619, accuracy val: 0.6228571428571429

In [ ]:
# accuracy train: 0.303452380952381, accuracy val: 0.30666666666666664

In [764]:
loss_train_all, loss_val_all = [], []
accuracy_train_all, accuracy_val_all = [], []

for epoch in range(NUM_EPOCHS):

    start = time.time()
    print(f'Epoch: {epoch}, Learning Rate: {scheduler.get_last_lr()[0]}\n')

    loss_epoch_train, loss_epoch_val = 0, 0
    correct_train, correct_val, total_train, total_val = 0, 0, 0, 0

    model.train()
    for (X_batch_train, y_batch_train) in tqdm(train_dl):

        X_batch_train, y_batch_train = X_batch_train.to(device), y_batch_train.to(device)

        optimizer.zero_grad()

        y_pred_train = model(X_batch_train)
        loss_train = criterion(y_pred_train, y_batch_train)

        loss_train.backward()
        optimizer.step()

        loss_epoch_train += loss_train.item()
        correct_train += accuracy(y_pred_train, y_batch_train)
        total_train += y_batch_train.size()[0]

    scheduler.step()
    model.eval()

    with torch.no_grad():
        
        for item in model.parameters():
            print(item.grad.mean())
        
        for (X_batch_val, y_batch_val) in tqdm(val_dl):

            X_batch_val, y_batch_val = X_batch_val.to(device), y_batch_val.to(device)

            y_pred_val = model(X_batch_val)
#                 print(y_pred_val)
            loss_val = criterion(y_pred_val, y_batch_val)

            loss_epoch_val += loss_val.item()

            correct_val += accuracy(y_pred_val, y_batch_val)
            total_val += y_batch_val.size()[0]


    loss_train_tmp = loss_epoch_train / len(train_dl)
    loss_val_tmp = loss_epoch_val / len(val_dl)

    loss_train_all.append(loss_train_tmp)
    loss_val_all.append(loss_val_tmp)

    accuracy_train_tmp = correct_train / total_train
    accuracy_val_tmp = correct_val / total_val

    accuracy_train_all.append(accuracy_train_tmp)
    accuracy_val_all.append(accuracy_val_tmp)

    end = time.time()
    print(f"epoch time: {end - start}")  
    print(f"loss train: {loss_train_tmp}, loss val: {loss_val_tmp}")
    print(f"accuracy train: {accuracy_train_tmp}, accuracy val: {accuracy_val_tmp}")

    print("---------------------------------------------------------------------------------------------------")

Epoch: 0, Learning Rate: 0.001




tensor(0.0001, device='cuda:3')
tensor(-3.6528e-06, device='cuda:3')
tensor(0.0001, device='cuda:3')
tensor(0.0001, device='cuda:3')
tensor(-9.8824e-06, device='cuda:3')
tensor(4.8760e-08, device='cuda:3')
tensor(0.0006, device='cuda:3')
tensor(0.0006, device='cuda:3')
tensor(1.1089e-05, device='cuda:3')
tensor(3.7611e-05, device='cuda:3')
tensor(-0.0013, device='cuda:3')
tensor(-0.0013, device='cuda:3')
tensor(-1.0288e-06, device='cuda:3')
tensor(0., device='cuda:3')
tensor(4.8747e-05, device='cuda:3')
tensor(4.8747e-05, device='cuda:3')
tensor(3.9633e-06, device='cuda:3')
tensor(-0.0013, device='cuda:3')
tensor(-2.6166e-09, device='cuda:3')
tensor(-2.8383e-09, device='cuda:3')



epoch time: 31.21553087234497
loss train: 2.1263486860376415, loss val: 1.71086024536806
accuracy train: 0.3319047619047619, accuracy val: 0.39714285714285713
---------------------------------------------------------------------------------------------------
Epoch: 1, Learning Rate: 0.001




tensor(0.0002, device='cuda:3')
tensor(1.7141e-05, device='cuda:3')
tensor(-0.0010, device='cuda:3')
tensor(-0.0010, device='cuda:3')
tensor(0.0001, device='cuda:3')
tensor(4.1071e-06, device='cuda:3')
tensor(-0.0005, device='cuda:3')
tensor(-0.0005, device='cuda:3')
tensor(-1.3853e-05, device='cuda:3')
tensor(3.4349e-05, device='cuda:3')
tensor(0.0015, device='cuda:3')
tensor(0.0015, device='cuda:3')
tensor(-1.5789e-06, device='cuda:3')
tensor(0., device='cuda:3')
tensor(8.0616e-05, device='cuda:3')
tensor(8.0616e-05, device='cuda:3')
tensor(1.1597e-05, device='cuda:3')
tensor(0.0009, device='cuda:3')
tensor(-2.3015e-09, device='cuda:3')
tensor(-1.9513e-09, device='cuda:3')



epoch time: 27.69304919242859
loss train: 1.6263731040737845, loss val: 1.5334508348913753
accuracy train: 0.4269047619047619, accuracy val: 0.4623809523809524
---------------------------------------------------------------------------------------------------
Epoch: 2, Learning Rate: 0.001



...


epoch time: 25.345417976379395
loss train: 0.07895822329870002, loss val: 0.22255438653861775
accuracy train: 0.9534523809523809, accuracy val: 0.9042857142857142
---------------------------------------------------------------------------------------------------
Epoch: 118, Learning Rate: 0.000125




tensor(3.3112e-05, device='cuda:3')
tensor(-3.3267e-06, device='cuda:3')
tensor(-0.0002, device='cuda:3')
tensor(-0.0002, device='cuda:3')
tensor(-5.3837e-06, device='cuda:3')
tensor(-2.6047e-07, device='cuda:3')
tensor(-0.0003, device='cuda:3')
tensor(-0.0003, device='cuda:3')
tensor(-2.7441e-07, device='cuda:3')
tensor(-2.1708e-06, device='cuda:3')
tensor(-0.0002, device='cuda:3')
tensor(-0.0002, device='cuda:3')
tensor(-7.5526e-08, device='cuda:3')
tensor(0., device='cuda:3')
tensor(2.0431e-05, device='cuda:3')
tensor(2.0431e-05, device='cuda:3')
tensor(-8.5615e-08, device='cuda:3')
tensor(9.6708e-06, device='cuda:3')
tensor(1.6487e-09, device='cuda:3')
tensor(1.0644e-09, device='cuda:3')



epoch time: 24.98110580444336
loss train: 0.07900763053277676, loss val: 0.2387387415065485
accuracy train: 0.9544047619047619, accuracy val: 0.9085714285714286
---------------------------------------------------------------------------------------------------
Epoch: 119, Learning Rate: 0.000125




tensor(2.9550e-05, device='cuda:3')
tensor(-6.8518e-06, device='cuda:3')
tensor(-0.0016, device='cuda:3')
tensor(-0.0016, device='cuda:3')
tensor(4.5583e-06, device='cuda:3')
tensor(-6.7629e-07, device='cuda:3')
tensor(-9.8186e-05, device='cuda:3')
tensor(-9.8186e-05, device='cuda:3')
tensor(-3.2512e-07, device='cuda:3')
tensor(-5.5816e-06, device='cuda:3')
tensor(-4.7412e-05, device='cuda:3')
tensor(-4.7412e-05, device='cuda:3')
tensor(-3.5400e-09, device='cuda:3')
tensor(0., device='cuda:3')
tensor(-6.4126e-06, device='cuda:3')
tensor(-6.4126e-06, device='cuda:3')
tensor(-2.8876e-06, device='cuda:3')
tensor(-7.2164e-05, device='cuda:3')
tensor(3.9308e-09, device='cuda:3')
tensor(3.0157e-09, device='cuda:3')



epoch time: 25.681734800338745
loss train: 0.0857430063188076, loss val: 0.21991805732250214
accuracy train: 0.9517857142857142, accuracy val: 0.909047619047619
---------------------------------------------------------------------------------------------------


# Доделать тензорборд

In [ ]:
# from torch.utils.tensorboard import SummaryWriter

In [ ]:
# default `log_dir` is "runs" - we'll be more specific here
# writer = SummaryWriter('runs/first_try')

In [ ]:
# writer.add_scalar('training loss',
#                             running_loss / 1000,
#                             epoch * len(trainloader) + i)

In [546]:
loss_train_all, loss_val_all = [], []
accuracy_train_all, accuracy_val_all = [], []

for epoch in range(NUM_EPOCHS):

    start = time.time()
    print(f'Epoch: {epoch}, Learning Rate: {scheduler.get_last_lr()[0]}\n')

    loss_epoch_train, loss_epoch_val = 0, 0
    correct_train, correct_val, total_train, total_val = 0, 0, 0, 0

    model.train()
    for (X_batch_train, y_batch_train) in tqdm(raw_dl):

        X_batch_train, y_batch_train = X_batch_train.to(device), y_batch_train.to(device)

        optimizer.zero_grad()

        y_pred_train = model(X_batch_train)
        loss_train = criterion(y_pred_train, y_batch_train)

        loss_train.backward()
        optimizer.step()

        loss_epoch_train += loss_train.item()
        correct_train += accuracy(y_pred_train, y_batch_train)
        total_train += y_batch_train.size()[0]

    scheduler.step()
    model.eval()

    with torch.no_grad():
        
        for item in model.parameters():
            print(item.grad.mean())
        
        for (X_batch_val, y_batch_val) in tqdm(raw_dl):

            X_batch_val, y_batch_val = X_batch_val.to(device), y_batch_val.to(device)

            y_pred_val = model(X_batch_val)
#                 print(y_pred_val)
            loss_val = criterion(y_pred_val, y_batch_val)

            loss_epoch_val += loss_val.item()

            correct_val += accuracy(y_pred_val, y_batch_val)
            total_val += y_batch_val.size()[0]


    loss_train_tmp = loss_epoch_train / len(train_dl)
    loss_val_tmp = loss_epoch_val / len(val_dl)

    loss_train_all.append(loss_train_tmp)
    loss_val_all.append(loss_val_tmp)

    accuracy_train_tmp = correct_train / total_train
    accuracy_val_tmp = correct_val / total_val

    accuracy_train_all.append(accuracy_train_tmp)
    accuracy_val_all.append(accuracy_val_tmp)

    end = time.time()
    print(f"epoch time: {end - start}")  
    print(f"loss train: {loss_train_tmp}, loss val: {loss_val_tmp}")
    print(f"accuracy train: {accuracy_train_tmp}, accuracy val: {accuracy_val_tmp}")

    print("---------------------------------------------------------------------------------------------------")

Epoch: 0, Learning Rate: 0.0005




tensor(5.4305e-05, device='cuda:3')
tensor(-1.4244e-06, device='cuda:3')
tensor(0.0001, device='cuda:3')
tensor(0.0001, device='cuda:3')
tensor(0.0006, device='cuda:3')
tensor(-3.2526e-07, device='cuda:3')
tensor(0.0025, device='cuda:3')
tensor(0.0025, device='cuda:3')
tensor(-1.4143e-05, device='cuda:3')
tensor(-3.0652e-05, device='cuda:3')
tensor(0.0014, device='cuda:3')
tensor(0.0014, device='cuda:3')
tensor(-1.5235e-07, device='cuda:3')
tensor(0., device='cuda:3')
tensor(1.0968e-05, device='cuda:3')
tensor(1.0968e-05, device='cuda:3')
tensor(-3.6101e-05, device='cuda:3')
tensor(0.0008, device='cuda:3')
tensor(-5.0994e-09, device='cuda:3')
tensor(-8.5149e-09, device='cuda:3')



epoch time: 73.42606234550476
loss train: 1.8815502858523168, loss val: 14.08593177094179
accuracy train: 0.4531428571428571, accuracy val: 0.4713333333333333
---------------------------------------------------------------------------------------------------
Epoch: 1, Learning Rate: 0.0005




tensor(0.0002, device='cuda:3')
tensor(-5.5606e-06, device='cuda:3')
tensor(0.0004, device='cuda:3')
tensor(0.0004, device='cuda:3')
tensor(0.0023, device='cuda:3')
tensor(2.6251e-06, device='cuda:3')
tensor(0.0098, device='cuda:3')
tensor(0.0098, device='cuda:3')
tensor(-3.0667e-05, device='cuda:3')
tensor(-0.0001, device='cuda:3')
tensor(0.0046, device='cuda:3')
tensor(0.0046, device='cuda:3')
tensor(-8.8504e-08, device='cuda:3')
tensor(0., device='cuda:3')
tensor(-3.2905e-06, device='cuda:3')
tensor(-3.2905e-06, device='cuda:3')
tensor(-8.2824e-05, device='cuda:3')
tensor(0.0021, device='cuda:3')
tensor(-2.2862e-09, device='cuda:3')
tensor(-2.8383e-09, device='cuda:3')



epoch time: 73.47364783287048
loss train: 1.8518035619547872, loss val: 14.031297269989462
accuracy train: 0.46390476190476193, accuracy val: 0.47695238095238096
---------------------------------------------------------------------------------------------------
Epoch: 2, Learning Rate: 0.0005



...


epoch time: 73.37758255004883
loss train: 1.7989739902091748, loss val: 13.705917386447682
accuracy train: 0.4796190476190476, accuracy val: 0.48638095238095236
---------------------------------------------------------------------------------------------------
Epoch: 18, Learning Rate: 0.00025




tensor(-0.0001, device='cuda:3')
tensor(7.3729e-06, device='cuda:3')
tensor(-0.0004, device='cuda:3')
tensor(-0.0004, device='cuda:3')
tensor(-0.0013, device='cuda:3')
tensor(-6.0784e-06, device='cuda:3')
tensor(-0.0052, device='cuda:3')
tensor(-0.0052, device='cuda:3')
tensor(9.2231e-06, device='cuda:3')
tensor(-2.5690e-05, device='cuda:3')
tensor(-0.0014, device='cuda:3')
tensor(-0.0014, device='cuda:3')
tensor(5.4327e-07, device='cuda:3')
tensor(0., device='cuda:3')
tensor(2.2863e-05, device='cuda:3')
tensor(2.2863e-05, device='cuda:3')
tensor(7.7831e-05, device='cuda:3')
tensor(-0.0021, device='cuda:3')
tensor(-1.1986e-09, device='cuda:3')
tensor(-1.7739e-09, device='cuda:3')



epoch time: 73.05480027198792
loss train: 1.798080257394097, loss val: 13.779789938646203
accuracy train: 0.4766666666666667, accuracy val: 0.4794285714285714
---------------------------------------------------------------------------------------------------
Epoch: 19, Learning Rate: 0.00025




tensor(0.0001, device='cuda:3')
tensor(-1.0550e-05, device='cuda:3')
tensor(0.0005, device='cuda:3')
tensor(0.0005, device='cuda:3')
tensor(-0.0004, device='cuda:3')
tensor(7.7219e-06, device='cuda:3')
tensor(-0.0016, device='cuda:3')
tensor(-0.0016, device='cuda:3')
tensor(1.0736e-05, device='cuda:3')
tensor(-1.4209e-05, device='cuda:3')
tensor(-0.0010, device='cuda:3')
tensor(-0.0010, device='cuda:3')
tensor(-5.0017e-07, device='cuda:3')
tensor(0., device='cuda:3')
tensor(3.6257e-05, device='cuda:3')
tensor(3.6257e-05, device='cuda:3')
tensor(3.8242e-06, device='cuda:3')
tensor(-0.0022, device='cuda:3')
tensor(2.3946e-10, device='cuda:3')
tensor(-3.5479e-10, device='cuda:3')



epoch time: 72.36645030975342
loss train: 2.049848956592155, loss val: 14.414148814537945
accuracy train: 0.44276190476190475, accuracy val: 0.4726666666666667
---------------------------------------------------------------------------------------------------


In [ ]:
# tensorboard --logdir=runs

In [ ]:
def train_model(model, optimizer, scheduler, train_dl, val_dl, device):

    loss_train_all, loss_val_all = [], []
    accuracy_train_all, accuracy_val_all = [], []

    for epoch in range(NUM_EPOCHS):

        start = time.time()
        print(f'Epoch: {epoch}, Learning Rate: {scheduler.get_last_lr()[0]}\n')

        loss_epoch_train, loss_epoch_val = 0, 0
        correct_train, correct_val, total_train, total_val = 0, 0, 0, 0

        model.train()
        for (X_batch_train, y_batch_train) in tqdm(train_dl):

            X_batch_train, y_batch_train = X_batch_train.to(device), y_batch_train.to(device)

            optimizer.zero_grad()

            y_pred_train = model(X_batch_train)
            loss_train = criterion(y_pred_train, y_batch_train)

            loss_train.backward()
            optimizer.step()

            loss_epoch_train += loss_train.item()
            correct_train += accuracy(y_pred_train, y_batch_train)
            total_train += y_batch_train.size()[0]

        scheduler.step()
        model.eval()

        with torch.no_grad():
            for (X_batch_val, y_batch_val) in tqdm(val_dl):

                X_batch_val, y_batch_val = X_batch_val.to(device), y_batch_val.to(device)

                y_pred_val = model(X_batch_val)
#                 print(y_pred_val)
                loss_val = criterion(y_pred_val, y_batch_val)

                loss_epoch_val += loss_val.item()

                correct_val += accuracy(y_pred_val, y_batch_val)
                total_val += y_batch_val.size()[0]


        loss_train_tmp = loss_epoch_train / len(train_dl)
        loss_val_tmp = loss_epoch_val / len(val_dl)

        loss_train_all.append(loss_train_tmp)
        loss_val_all.append(loss_val_tmp)

        accuracy_train_tmp = correct_train / total_train
        accuracy_val_tmp = correct_val / total_val

        accuracy_train_all.append(accuracy_train_tmp)
        accuracy_val_all.append(accuracy_val_tmp)

        end = time.time()
        print(f"epoch time: {end - start}")  
        print(f"loss train: {loss_train_tmp}, loss val: {loss_val_tmp}")
        print(f"accuracy train: {accuracy_train_tmp}, accuracy val: {accuracy_val_tmp}")

        print("---------------------------------------------------------------------------------------------------")

In [391]:
# train_model(model, optimizer, scheduler, raw_dl, raw_dl, device)

In [ ]:
# 0.6666666666666666, accuracy val: 0.6666666666666666

In [190]:
item

Parameter containing:
tensor([-0.0342,  0.0146, -1.2723, -1.1949, -1.2070, -1.2265, -1.3617, -1.3242,
        -1.1334, -1.1038, -1.1241, -1.3348, -1.0677, -1.2345, -1.0239, -1.2438,
        -1.1649, -1.1856, -1.2729, -1.2864, -0.0018], device='cuda:3',
       requires_grad=True)

In [192]:
item.grad.mean()

tensor(3.5479e-10, device='cuda:3')

In [188]:
for item in model.parameters():
    print(item.grad)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:3')
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:3')
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:3')
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:3')
tensor([[-5.0143e-06,  0.0000e+00,  2.5366e-06,  ...,  0.0000e+00,
          1.8329e-17,  0.0000e+00],
        [ 3.4783e-08,  0.0000e+00,  1.5160e-08,  ...,  0.0000e+00,
          3.7292e-20,  0.0000e+00],
        [ 1.0741e-06,  0.0000e+00, -7.4472e-07,  ...,  0.0000e+00,
          2.7790e-17,  0.0000e+00],
        ...,
        [-2.0572e-08,  0

In [ ]:
plt.figure(figsize=(20, 8))
plt.title("loss")
plt.plot(np.arange(len(loss_train_all)), loss_train_all, '-o', marker='.', label='train')
plt.plot(np.arange(len(loss_val_all)), loss_val_all, '-o', marker='.', label='val')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20, 8))
plt.title("accuracy")
plt.plot(np.arange(len(accuracy_train_all)), accuracy_train_all, '-o', marker='.', label='train')
plt.plot(np.arange(len(accuracy_val_all)), accuracy_val_all, '-o', marker='.', label='val')
plt.legend()
plt.show()

# ----------------------------------------------------------------------------------------